# On Calculating the Gaussian Mean Width of Finite Sets
Unfortunately, GitHub currently seems unable to render the markdown formulas correctly. For the correctly rendered formulas we refer to [html](https://github.com/sdittmer/On_Calculating_the_Gaussian_Mean_Width_of_Finite_Sets/blob/master/On_Calculating_the_Gaussian_Mean_Width_of_Finite_Sets.ipynb).
## What are we doing?
We aim to calculate/approximate the Gaussian mean with of a finite set $K\subset\mathbb{R}^n$.

## What is the Gaussian Mean Width?
The **Gaussian mean width** ([Vershynin](https://arxiv.org/abs/1405.5103)) of a set $K\subset\mathbb{R}^n$ is defined as

$$\omega(K):=\mathbb{E}_{g\sim\mathcal{N}(0,1_n)} \sup_{x\in K-K} \left<g, x\right>,$$
here $\mathcal{N}(0,1_n)$ denotes an n-dimensional standard Gaussian i.i.d. vector and $K-K$ is to be read in the Minkowski sense.

## Approach:
As argued in ([Vershynin](https://arxiv.org/abs/1405.5103)), due to the Gaussian concentration of measure,
$$\sup_{x\in K-K} \left<g, x\right>$$
for one $g\sim\mathcal{N}(0,1_n)$ already yields a good estimate for $\omega(K)$. In practice, to make this estimate more stable, we averaged over the results of 100 samples of $g$. Due to the invariance of the Gaussian mean with under convexification we can replace the Formula above by 
$$\sup_{x\in Conv(K)-Conv(K)} \left<g, x\right>,$$
where $Conv(K)$ denotes the convex hull of $K$.

## Algorithm
In what follows, we use the notation $:$ to represent concatenations of the vectors/matrices and. Additionally we will overload the notation for the set $K=\{x_1,\dots,x_{|K|}\}$ to also represent a matrix $K\in\mathbb{R}^{|K|\times n}$ where each row is given by a different element of the set $K$. This allows us to formulate the solution of Formula above via the constraint optimization problem

$$\max_x\left<g, x\right>, \textrm{s.t.}$$

$$
\begin{pmatrix}
1 & \cdots & 1 & 0 & \cdots & 0 \\
0 & \cdots & 0 & 1 & \cdots & 1
\end{pmatrix} 
\begin{pmatrix} 
\alpha  \\
:\\
\beta 
\end{pmatrix}
=
\begin{pmatrix} 
1  \\ 1 
\end{pmatrix},
$$

$$(\alpha:-\beta)\ge0 \textrm{ and } (K^T:K^T)(\alpha:-\beta)^T=x,$$
where $\alpha, \beta \in \mathbb{R}^{|K|}$.

Rewriting yields the algorithm in form of the following linear program:
$$
\min_{(\alpha:-\beta)}
\left<
-
\begin{pmatrix} 
K\\
:\\
K 
\end{pmatrix}
g,
\begin{pmatrix} 
\alpha  \\
:\\
\beta 
\end{pmatrix}
\right> \textrm{ s.t. the first two requirements above hold}.
$$

In [1]:
import numpy as np
from scipy.optimize import linprog
from scipy.stats import trim_mean
import matplotlib.pyplot as plt

In [2]:
def mean_width_via_linprog(K, eta = None, gaussian=True):
    """
    Estimates the mean width of the set K via the calculation of
    sup <eta, z> for z in Conv(K-K).

    Keyword arguments:
    K -- A matrix, where each column is representing an element of the set K.
    eta -- Gaussian/unit vector (default random Gaussian/unit vector)
    gaussian -- whether Gaussian or Spherical mean width 
    """
    K = K.T
    m, n = K.shape
    if eta is None:
        eta = np.random.normal(0, 1, n)
    if not gaussian:
        eta = eta / np.linalg.norm(eta)

    K_over_K = np.vstack((K, K))

    # only non-negative entries
    bounds = (0, None)
    A_eq = np.vstack((
                        np.hstack((np.ones(m), np.zeros(m))),
                        np.hstack((np.zeros(m), np.ones(m)))
                    ))
    b_eq = np.ones(2)
    c = - np.dot(K_over_K, eta)

    res = linprog(c=c, A_eq=A_eq, b_eq=b_eq, bounds=(bounds))
    delta = res['x']

    x = np.dot(K_over_K.T, delta)
    sup = np.dot(x, eta)
    return sup

In [3]:
def get_mean_width_estimate(K,
                             aggregate_over_n_estimations=100,
                             gaussian=True):
    """
    Estimates the mean width of the set K via the usage of
    multiple instances of mean_width_via_linprog to get a
    better estimate via calculating a trimmed mean.

    Keyword arguments:
    K -- A matrix, where each column is representing an element of the set K.
    aggregate_over_n_estimations -- number of instances over which we calculate the timmed mean (default 100)
    gaussian -- whether Gaussian or Spherical mean width
    """
    estimation_s = []
    for _ in range(aggregate_over_n_estimations):
        estimation = mean_width_via_linprog(K, eta = None, gaussian=gaussian)
        estimation_s += [estimation]

    estimation_s = np.array(estimation_s)

    estimation = trim_mean(estimation_s, 0.1)

    return estimation

# Simple Example
As an example we want to calculate the Gaussian mean width of a set $K$ given by $1,000$ random vectors sampled from $\mathcal{N}(0,1_5)$.

In [5]:
# We create the example set.
K = np.random.normal(0, 1, [5, 1000])
# We calaculate the Gaussian mean width
gmw = get_mean_width_estimate(K)
# Print result
print(f"The Gaussian mean width of the set K is approximately {gmw}.")

The Gaussian mean width of the set K is approximately 13.324691231725575.
